In [1]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from helpers import *

In [4]:
#Create embeddings with glove_solution
from glove_solution import main as glove_main
glove_main()

loading cooccurrence matrix
6496907 nonzero entries
using nmax = 100 , cooc.max() = 207302
initializing embeddings
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [2]:
from pattern.en import *

In [3]:
word_embeddings = np.load("embeddings.npy")
word_embeddings.shape

(21161, 20)

In [4]:
words_list = words_list("vocab_cut.txt")

In [5]:
tweets_pos_txt = tweets_txt("Datasets/twitter-datasets/train_pos.txt")
tweets_neg_txt = tweets_txt("Datasets/twitter-datasets/train_neg.txt")

# Do some data cleaning

In [6]:
#Remove duplicates
unique_tweets_pos_txt = remove_duplicated_tweets_txt(tweets_pos_txt)
unique_tweets_neg_txt = remove_duplicated_tweets_txt(tweets_neg_txt)

In [7]:
#Spelling changes
spelling_dict = {"u" : "you", "dont" : "don't", "cant" : "can't", "r" : "are", "wont" : "won't"}

In [9]:
#Filter tweets
common = ['"', ',', '.', ')', '(', '-', \
                "<user>", "<url>", "a", "the", "of", "to", \
                "it", "this", "that", "these", "there"]
tweets_pos_vec_clean = tweet_means(unique_tweets_pos_txt, word_embeddings,
                             words_list, word_embeddings[0].shape[0],
                             spelling=True, spelling_dict=spelling_dict, 
                             negation=True, 
                             clean=True, common=common)
tweets_neg_vec_clean = tweet_means(unique_tweets_neg_txt, word_embeddings, 
                             words_list, word_embeddings[0].shape[0], 
                             True, spelling_dict, 
                             True, 
                             True, common)

np.save('tweets_pos_vec_clean', tweets_pos_vec_clean)
np.save('tweets_neg_vec_clean', tweets_neg_vec_clean)

In [ ]:
#Means for tweets not filtered
tweets_pos_vec = tweet_means(tweets_pos_txt, \
    word_embeddings, words_list, word_embeddings[0].shape[0])
tweets_neg_vec = tweet_means(tweets_new_txt, \
    word_embeddings, words_list, word_embeddings[0].shape[0])

np.save('tweets_pos_vec', tweets_pos_vec)
np.save('tweets_neg_vec', tweets_neg_vec)

In [ ]:
np.save('tweets_pos_vec', tweets_pos_vec)
np.save('tweets_neg_vec', tweets_neg_vec)

# TRAINING TIME

In [6]:
tweets_pos_vec = np.load('tweets_pos_vec_clean.npy')
tweets_neg_vec = np.load('tweets_neg_vec_clean.npy')

In [ ]:
#TO USE IF ALGO ASKS LABEL TO BE 1 AND -1
X = np.concatenate((tweets_pos_vec, tweets_neg_vec))
y = np.concatenate((np.ones((tweets_pos_vec.shape[0])), -1*np.ones((tweets_neg_vec.shape[0]))))

In [7]:
#TO USE IF ALGO ASKS LABEL TO BE 1 AND 0
X = np.concatenate((tweets_pos_vec, tweets_neg_vec))
y = np.concatenate((np.ones((tweets_pos_vec.shape[0])), np.zeros((tweets_neg_vec.shape[0]))))

In [ ]:
#NEED TO CHECK IF IT IS REALLY WHAT SCORING ASKS
def accuracy(estimator, X, y):
    return np.mean(y==estimator.predict(X))

In [ ]:
numpy.random.seed(42)
clf = ???
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
scores = cross_val_score(estimator=clf, X, y, scoring=accuracy, cv=cv)

In [9]:
# VERSION SAMUEL : 

j = 0.9

indices = np.random.permutation(len(y))

X = X[indices]
y = y[indices]

train_X = X[0:int(j*len(y))]
train_y = y[0:int(j*len(y))]
test_X = X[int(j*len(y)):]
test_y = y[int(j*len(y)):]

indices = np.random.permutation(len(y))

X = X[indices]
y = y[indices]

def accuracy(y1, y2):
    return np.mean(y1==y2)

lsvc = svm.LinearSVC(max_iter=10000)
lsvc.fit(train_X, train_y)
print("SVM : " + str(accuracy(test_y, lsvc.predict(test_X))))

sgdcl = SGDClassifier()
sgdcl.fit(train_X, train_y)
print("SGD : " + str(accuracy(test_y, sgdcl.predict(test_X))))

lr = LogisticRegression()
lr.fit(train_X, train_y)
print("LR : " + str(accuracy(test_y, lr.predict(test_X))))

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
mlp.fit(train_X, train_y)
print("MLP : " + str(accuracy(test_y, mlp.predict(test_X))))

SVM : 0.6134671593227816
SGD : 0.6115921248552363


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR : 0.6081177962830199
MLP : 0.6166105994595489


In [ ]:
# Accuracy after clean + Samuel pipeline

SVM : 0.6134671593227816
SGD : 0.6115921248552363
LR : 0.6081177962830199
MLP : 0.6166105994595489